# 1

In [108]:
import pandas as pd
import numpy as np
num_users = 5
num_items = 8
utility =  None
utility_original = None
utility = pd. DataFrame (np.nan,
                         index =[f"user-{i}" for i in range (1, num_users +1)],
                         columns =[f"song-{j}" for j in range (1, num_items +1) ])


artists = {
"artist-1": ["song-1", "song-2", "song-3"],
"artist-2": ["song-4"],
"artist-3": ["song-5", "song-6"],
"artist-4": ["song-7", "song-8"]
}
user_likes = """
user-1 song-1 5
user-1 song-4 4
user-1 song-5 1
user-1 song-6 1
user-2 song-2 1
user-2 song-6 5
user-2 song-7 4
user-2 song-8 2
user-3 song-3 2
user-3 song-4 5
user-3 song-6 2
user-4 song-1 2
user-4 song-5 5
user-4 song-2 2
user-5 song-7 1
user-5 song-2 5
user-5 song-3 3
user-5 song-4 5
"""
for user_id , song , rating in [line.split(" ") for line in filter(lambda line:line.strip() != "", user_likes.strip().split("\n"))]:
    rating = np.float64( rating )
    utility.at[ user_id , song ] = rating
utility_original = utility.copy()
display(utility)


,song-1,song-2,song-3,song-4,song-5,song-6,song-7,song-8
user-1,5.0,NaN,NaN,4.0,1.0,1.0,NaN,NaN
user-2,NaN,1.0,NaN,NaN,NaN,5.0,4.0,2.0
user-3,NaN,NaN,2.0,5.0,NaN,2.0,NaN,NaN
user-4,2.0,2.0,NaN,NaN,5.0,NaN,NaN,NaN
user-5,NaN,5.0,3.0,5.0,NaN,NaN,1.0,NaN


In [109]:
utility = utility.replace(np.nan, 0)
utility = utility[utility != 0].sub(utility.mean(axis=1), axis=0)
utility = utility.replace(np.nan, 0)
print(utility)

        song-1  song-2  song-3  song-4  song-5  song-6  song-7  song-8
user-1   3.625   0.000   0.000   2.625  -0.375  -0.375    0.00     0.0
user-2   0.000  -0.500   0.000   0.000   0.000   3.500    2.50     0.5
user-3   0.000   0.000   0.875   3.875   0.000   0.875    0.00     0.0
user-4   0.875   0.875   0.000   0.000   3.875   0.000    0.00     0.0
user-5   0.000   3.250   1.250   3.250   0.000   0.000   -0.75     0.0


In [110]:
cos_sim = lambda a,b : (a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b))

In [111]:
cos_sim(utility.iloc[0], utility.iloc[1])

-0.06680990011245666

In [112]:
cos_sim(utility.iloc[0], utility.iloc[2])

0.5369339849200522

The taste of U1 and U2 are at -0.066 cos, so not very similiar. U1 and U3 are more similiar with a cos_sim of 0.536

In [136]:
def recommend(u_id, i_id, um, m=3):
    sims = []
    sums=None
    for i,_ in enumerate(um.index):
        if i == u_id:
            continue
        sims.append((i, (cos_sim(utility.iloc[u_id], utility.iloc[i]))))
    sims = np.sort(np.array(sims, dtype=[("x",int), ("y", float)]), order="y")[::-1][:m]
    for user,sim in sims:
        if sums == None:
            sums = sim * utility.iloc[user][i_id]
        else:
            sums += sim * utility.iloc[user][i_id]
    return (1/m) * sums

recommend(4,5,utility,m = 2)

0.2316968765669151

In [140]:
print("User-5 will like song-6 with 4 recommenders", recommend(4,5,utility,m = 4))
print("User-5 will like song-6 with 4 recommenders", recommend(4,0,utility,m = 4))

User-5 will like song-6 with 3 recommenders -0.029861045062920658
User-5 will like song-6 with 3 recommenders 0.3874839089792117


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'